# Perspectives in Computational Research: Homework 02

In [5]:
#Libraries
import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
from keras.utils import to_categorical
from keras import models
from keras import layers
from keras import losses
from keras import metrics

### Load Data

In [3]:
# Train Data
train_data = pd.read_csv('data/congress_train.csv', encoding = "ISO-8859-1")
train_title = train_data['Title'].tolist()
train_major = train_data['Major']

# Validata Data
val_data = pd.read_csv('data/congress_val.csv', encoding = "ISO-8859-1")
val_title = val_data['Title'].tolist()
val_major = val_data['Major']

# Test Data
test_data = pd.read_csv('data/congress_test.csv', encoding = "ISO-8859-1")
test_title = test_data['Title'].tolist()
test_major = test_data['Major']

### Tokenize

In [4]:
token = Tokenizer(num_words=10000) #class restricting to 10000 words

In [ ]:
# Train Data
token.fit_on_texts(train_title)
train_sequence = token.text_to_word_sequences(train_title)
train_x = pad_sequences(train_sequence, maxlen=100)
train_y = to_categorical(train_major)

In [ ]:
# validate Data
token.fit_on_texts(val_title)
val_sequence = token.text_to_word_sequences(val_title)
val_x = pad_sequences(val_sequence, maxlen=100)
val_y = to_categorical(val_major)

In [ ]:
# validate Data
token.fit_on_texts(test_title)
test_sequence = token.text_to_word_sequences(test_title)
test_x = pad_sequences(test_sequence, maxlen=100)
test_y = to_categorical(test_major)